In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import numpy as np

num_classes = 3
emotion_classes = ['No Face', 'Multiple Face','Blurred Image']

batch_size = 16
epochs = 9
input_shape = (224, 224, 3)
learning_rate = 0.0001  # Reduced learning rate for potentially better convergence

model = keras.Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=input_shape),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),  # Increased the number of units
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=30,  # Increased rotation range
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,  # Added shear range
    zoom_range=0.2,  # Added zoom range
    horizontal_flip=True,
    fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Datasets/Face_No_Face_Multiple_face_blurred_dataset/train',
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/Datasets/Face_No_Face_Multiple_face_blurred_dataset/test',
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'
)

model.fit(
    train_generator,
    epochs=epochs,
    verbose=1,
    validation_data=test_generator
)

save_path = '/content/drive/MyDrive/Datasets1/noface_final.keras'
model.save(save_path)


Found 321 images belonging to 3 classes.
Found 193 images belonging to 3 classes.
Epoch 1/9
21/21 [==============================] - 250s 11s/step - loss: 0.9140 - accuracy: 0.6604 - val_loss: 0.8998 - val_accuracy: 0.6580
Epoch 2/9
21/21 [==============================] - 153s 7s/step - loss: 0.8369 - accuracy: 0.6791 - val_loss: 0.7970 - val_accuracy: 0.6580
Epoch 3/9
21/21 [==============================] - 176s 8s/step - loss: 0.8075 - accuracy: 0.6822 - val_loss: 0.7515 - val_accuracy: 0.6528
Epoch 4/9
21/21 [==============================] - 154s 7s/step - loss: 0.7490 - accuracy: 0.6978 - val_loss: 0.7360 - val_accuracy: 0.6736
Epoch 5/9
21/21 [==============================] - 153s 7s/step - loss: 0.7365 - accuracy: 0.7072 - val_loss: 0.7083 - val_accuracy: 0.6373
Epoch 6/9
21/21 [==============================] - 172s 8s/step - loss: 0.7197 - accuracy: 0.7009 - val_loss: 0.6976 - val_accuracy: 0.6580
Epoch 7/9
21/21 [==============================] - 186s 9s/step - loss: 0.715

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
save_path = '/content/drive/MyDrive/Datasets1/noface_final.h5'
model.save(save_path)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
save_path = '/content/drive/MyDrive/models/noface_final.h5'
model.save(save_path)

In [ ]:
example_image_path = '/content/drive/MyDrive/Datasets/Face_No_Face_Multiple_face_blurred_dataset/validation/blurred/WhatsApp Image 2023-12-17 at 2.10.58 PM.jpeg'
img = keras.preprocessing.image.load_img(example_image_path, target_size=input_shape[:2])
img = keras.preprocessing.image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img /= 255.0


predictions = model.predict(img)

for i, emotion in enumerate(emotion_classes):
    print(f'Predicted {emotion}: {predictions[0][i] * 100:.2f}%')

1/1 [==============================] - 0s 138ms/step
Predicted No Face: 4.45%
Predicted Multiple Face: 26.52%
Predicted Blurred Image: 69.04%


In [ ]:
example_image_path = '/content/drive/MyDrive/Datasets/Face_No_Face_Multiple_face_blurred_dataset/validation/Multiple_face/WhatsApp Image 2023-12-16 at 12.59.39 PM.jpeg'
img = keras.preprocessing.image.load_img(example_image_path, target_size=input_shape[:2])
img = keras.preprocessing.image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img /= 255.0


predictions = model.predict(img)

for i, emotion in enumerate(emotion_classes):
    print(f'Predicted {emotion}: {predictions[0][i] * 100:.2f}%')

1/1 [==============================] - 0s 216ms/step
Predicted No Face: 27.61%
Predicted Multiple Face: 71.39%
Predicted Blurred Image: 1.00%


In [3]:
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib

In [5]:
noface_model = keras.models.load_model('/content/drive/MyDrive/models/noface_final.h5')

In [8]:
example_image_path = '/content/drive/MyDrive/Datasets/Face_No_Face_Multiple_face_blurred_dataset/validation/blurred/WhatsApp Image 2023-12-17 at 2.10.58 PM.jpeg'
input_shape = (224, 224, 3)
num_classes = 3
emotion_classes = ['No Face', 'Multiple Face','Blurred Image']
img = keras.preprocessing.image.load_img(example_image_path, target_size=input_shape[:2])
img = keras.preprocessing.image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img /= 255.0


predictions = noface_model.predict(img)

for i, emotion in enumerate(emotion_classes):
    print(f'Predicted {emotion}: {predictions[0][i] * 100:.2f}%')

1/1 [==============================] - 0s 132ms/step
Predicted No Face: 4.45%
Predicted Multiple Face: 26.52%
Predicted Blurred Image: 69.04%


In [10]:
example_image_path = '/content/drive/MyDrive/Datasets/Face_No_Face_Multiple_face_blurred_dataset/validation/Multiple_face/WhatsApp Image 2023-12-16 at 12.59.39 PM.jpeg'
img = keras.preprocessing.image.load_img(example_image_path, target_size=input_shape[:2])
img = keras.preprocessing.image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img /= 255.0


predictions = noface_model.predict(img)

for i, emotion in enumerate(emotion_classes):
    print(f'Predicted {emotion}: {predictions[0][i] * 100:.2f}%')

1/1 [==============================] - 0s 138ms/step
Predicted No Face: 27.61%
Predicted Multiple Face: 71.39%
Predicted Blurred Image: 1.00%
